In [13]:
# generatize image grid:  image lowpass adain adain&lowpass blockadain blockadain&lowpass
import torchvision
import torch
import torchvision.utils as vutils

def calc_mean_std(feat, eps=1e-5):
    # eps is a small value added to the variance to avoid divide-by-zero.
    size = feat.size()
    # assert (len(size) == 4)
    other = size[:-2]
    feat_var = feat.reshape(*other, -1).var(dim=-1) + eps
    feat_std = feat_var.sqrt().reshape(*other, 1, 1)
    feat_mean = feat.reshape(*other, -1).mean(dim=-1).reshape(*other, 1, 1)
    return feat_mean, feat_std


def adaIN(content_feat, style_feat):
    assert (content_feat.size()[:-2] == style_feat.size()[:-2])
    size = content_feat.size()
    style_mean, style_std = calc_mean_std(style_feat)
    content_mean, content_std = calc_mean_std(content_feat)

    normalized_feat = (content_feat - content_mean) / content_std
    return normalized_feat * style_std + style_mean


def block_adaIN(content_feat, style_feat, block = 16):
    assert (content_feat.size()[:-2] == style_feat.size()[:-2])
    content_feat = blockzation(content_feat, block)
    style_feat = blockzation(style_feat, block)
    return  unblockzation(adaIN(content_feat, style_feat))

def blockzation(feat, block = 16):
    H, W = feat.size()[-2:]
    assert H % block == 0
    assert W % block == 0
    size = feat.size()[:-2]
    feat = feat.reshape(*size, H // block, block, W // block, block).transpose(-2, -3)
    return feat

def unblockzation(feat):
    size = feat.size()
    H = size[-4] * size[-2]
    W = size[-3] * size[-1]
    size = size[:-4]
    return feat.transpose(-2, -3).reshape(*size, H, W)

namelist=["00000.png", "00001.png", "00002.png", "00003.png"]
another = ["00001.png","00005.png","00009.png","00013.png"]
anotherl = ["lowdiv/", "energetest/", "naive_target_new_difblock/"]
source =["flickr_cat_000056.jpg","flickr_cat_000108.jpg","flickr_cat_000136.jpg", "flickr_cat_000183.jpg"]
pathlist=[ "tmp/", "adaandlow/", "adablock/", "adablocklow/","energetest3/","energetest/","energetest6/"]
images =[]
img = torchvision.io.read_image("ref_imgs/cat/flickr_cat_000108.jpg")
# images.append(img)
img = blockzation(img.float(), block=32)
img2,_ = calc_mean_std(img)
print(img2.shape)
img2 = unblockzation(img2 * torch.ones([3,16,16, 16, 16]))
torchvision.io.write_png(torch.tensor(img2,dtype=torch.uint8),"blockmean.png")
for it in anotherl:
    for iit in namelist:
        images.append(torchvision.io.read_image(it+iit))

# img = torch.stack(images, 0)

img = vutils.make_grid(images,4,3)
torchvision.io.write_png(img,"grid3.png")

torch.Size([3, 16, 16, 1, 1])


/home/sunsk/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
